<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Открытие-файла-и-ознакомление-с-ним" data-toc-modified-id="Открытие-файла-и-ознакомление-с-ним-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Открытие файла и ознакомление с ним</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Инструкция по выполнению проекта**<p>1. Загрузите и изучите данные.<p>2. Ответьте на вопрос и обоснуйте решение.<p>Признаки умножают на обратимую матрицу.<p>Изменится ли качество линейной регрессии? (Её можно обучить заново.)<p>a. Изменится. Приведите примеры матриц.<p>b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.<p>3. Предложите алгоритм преобразования данных для решения задачи.<p>Обоснуйте, почему качество линейной регрессии не поменяется.<p>4. Запрограммируйте этот алгоритм, применив матричные операции.<p>Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

**Описание данных**

Набор данных находится в файле /datasets/insurance.csv
- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

Импорт необходимых для исследования инструментов

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

### Открытие файла и ознакомление с ним

In [ ]:
df = pd.read_csv('/datasets/insurance.csv')
display(df.sample(5))
df.info()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
2080,1,27.0,41800.0,1,0
3621,1,33.0,39800.0,0,0
2524,1,41.0,34100.0,0,0
2685,1,24.0,50000.0,0,0
1663,0,29.0,40300.0,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Проверка уникальных значений каждого признака

In [ ]:
df['Пол'].value_counts()

0    2505
1    2495
Name: Пол, dtype: int64

In [ ]:
df['Члены семьи'].value_counts()

1    1814
0    1513
2    1071
3     439
4     124
5      32
6       7
Name: Члены семьи, dtype: int64

In [ ]:
df['Страховые выплаты'].value_counts()

0    4436
1     423
2     115
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

Проверка на наличие явных дубликатов строк

In [ ]:
df.duplicated().sum()

153

Преобразование типов данных *float* в *int*. Построение матрицы корреляции признаков.

In [ ]:
df['Возраст'] = df['Возраст'].astype('int')
df['Зарплата'] = df['Зарплата'].astype('int')
df.info()
df.corr()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int64
 1   Возраст            5000 non-null   int64
 2   Зарплата           5000 non-null   int64
 3   Члены семьи        5000 non-null   int64
 4   Страховые выплаты  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


### Вывод

- датафрейм состоит из четырёх признаков и одного целевого, в которых по 5 тысяч строк
- пропусков в данных не обнаружено
- распределение значений признаков неравномерное
- присутствуют явные дубликаты, но нашему исследованию они не помешают
- все типы данных приведены к целочисленным
- матрица корелляции показывает прямую связь между возрастом клиента и количеством страховых выплат. Среди других признаков зависимостей не прослеживается.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** качество линейной регрессии не изменится

**Обоснование:** 

Предсказания для новой матрицы:
$$
a_P = XPw
$$

Вектор весов:
$$
w = ((XP)^T (XP))^{-1} (XP)^T y
$$

Подставим в формулу расчёта предсказаний значение вектора весов:
$$
a_P = (XP)((XP)^T (XP))^{-1} (XP)^T y
$$

Раскроем скобки с транспонированием:
$$
a_P = (XP)(P^TX^TXP)^{-1}P^TX^T y
$$

Выносим ***P*** за скобки, так как она квадратная (по условию):
$$
a_P = XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$

Ввиду того, что выражения:
$$
PP^{-1}
$$
и
$$
P^T(P^T)^{-1}
$$
равны ***E***, получаем:

$$
a_P = X(X^T X)^{-1}X^T y
$$

Из чего следует:
$$
a_P= Xw=a
$$

## Алгоритм преобразования

**Алгоритм**

1. Деление данных на две выборки.
2. Обучение модели и вычисление коэффициента детерминации.
3. Умножение признаков на обратимую матрицу.
4. Обучение модели и вычисление коэффициента детерминации на изменённых данных.
5. Сравнение коэффициентов детерминации моделей.

**Обоснование**

Качество линейной регрессии не изменится, потому что веса признаков до и после преобразования будут равны.

## Проверка алгоритма

Внесение признаков в отдельные переменные.

In [ ]:
features = df.drop('Страховые выплаты',axis=1)
target = df['Страховые выплаты']

Деление данных на обучающую и тестовую выборки.

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=.25, random_state=12345)

Обучение модели линейной регрессии.

In [ ]:
model = LinearRegression()
model.fit(features_train, target_train)

LinearRegression()

Вычисление коэффициента детерминации.

In [ ]:
r2_base = r2_score(target_test, model.predict(features_test))
'Базовый коэффициент детерминации', r2_base

('Базовый коэффициент детерминации', 0.4352275684083322)

Создание обратимой матрицы для кодирования.

In [ ]:
matrix_encoder = np.random.normal(size=(features_train.shape[1], features_train.shape[1]))

Проверка полученной матрицы на обратимость: найдём для неё обратную матрицу.

In [ ]:
matrix_encoder_inverse = np.linalg.inv(matrix_encoder)
matrix_encoder_inverse

array([[-2.18567465, -4.25557765, -0.64882818, -1.99152845],
       [ 0.1510414 ,  0.29376766, -0.38635327, -0.22289231],
       [-0.71042662, -2.47373698, -0.22580772, -1.58395271],
       [-2.47030496, -6.21695178, -1.10808186, -3.0149389 ]])

Умножение признаков на обратимую матрицу.

<div class="alert alert-info"> <b>ℹ️ Комментарий студента: </b> Спасибо. Беру на вооружение:) </div>

In [ ]:
features_train_encoded = features_train.values@matrix_encoder
features_test_encoded = features_test.values@matrix_encoder
features_train_encoded

array([[ 25553.80121585, -50287.99565488,  52663.47584724,
        -40829.93865579],
       [ 40622.49381378, -80047.73399605,  83779.57175363,
        -64930.57229903],
       [ 28995.36046567, -57110.63170678,  59782.74042043,
        -46339.11448754],
       ...,
       [ 31540.47298194, -62107.36035205,  65022.85950016,
        -50403.88424304],
       [ 35321.79975294, -69636.26554846,  72859.5342266 ,
        -56463.18017381],
       [ 28769.70784034, -56705.09582712,  59340.98172576,
        -45987.62223029]])

Обучение модели линейной регрессии.

In [ ]:
model_encoded = LinearRegression()
model_encoded.fit(features_train_encoded, target_train)

LinearRegression()

Вычисление коэффициента детерминации для модели шифрованных данных.

In [ ]:
r2_encode = r2_score(target_test, model_encoded.predict(features_test_encoded))
'Коэффициент детерминации модели на зашифрованных данных', r2_encode

('Коэффициент детерминации модели на зашифрованных данных',
 0.43522756840756316)

## Общий вывод

В процессе выполнения проекта по защите данных клиентов страховой компании проделана следующая работа:
- Проанализированы предоставленные данные: они представляют из себя датафрейм пяти признаков, в каждом из которых 5000 значений.
- Для шифрования данных применено их умножение на обратимую матрицу.
- Для построения предсказаний о том, обратится ли потенциальный клиент за выплатами в компанию, использована модель линейной регрессии.
- Для проверки уровня изменения качества проведены измерения коэффициентов детерминации модели до и после преобразований - они оказались равными, что свидетельствует о том, что выбор варианта шифрования не сказался на качестве модели.